# Multispectral Remote Sensing of Urban Green Spaces - A Case Study of Associated Neighborhoods in Boise, ID

## Site Description - Boise, Idaho

The city of Boise is located in a semi-arid climatic zone in the western Snake River Plain region of west-central Idaho. On average, the city receives only eleven inches of precipitation annually [(National Weather Service, 2023)](https://www.weather.gov/boi/climatesummary#:~:text=The%20semi%2Darid%20climate%20of,warm%20season%20is%20mostly%20dry.). The majority of the drinking and irrigation water feeding into Boise comes from trapped moisture and snow-melt from the mountains stored in a series of dams and canals on the Idaho and Snake Rivers.

![Dams of Boise](https://www.usbr.gov/pn/snakeriver/dams/maps/boise.jpg)

**Map of major rivers and dams providing water to the western Snake River Plain [(Bureau of Reclamation, 2020)](https://www.usbr.gov/pn/snakeriver/dams/middlesnake/boise.html)**


Recognizing both the scaricity of water but equal need for environmental and recreational greenspaces, the city of Boise, along with many partners, has been working since 2002 to develop environmentally conscious green spaces that meet the needs of the region's growing population, businesses, and native biodiversity. The city currently owns and manages 14 environmental reserves totaling nearly 5,000 acres and over 210 miles of trails [(City of Boise, 2023)](https://www.cityofboise.org/departments/parks-and-recreation/open-space/). And while the abundance of parks and green spaces already developed in the city of Boise benefit the health and wellness of residents, businesses, and communities, it remains unclear how widely the principle of green spaces is adopted within Boise's associated neighborhoods.

In [1]:
# Import standard libraries
import logging              # Logging of messages
import os                   # Handles files and directories
import shutil               # Clear cashe
import warnings             # Show warning triggered by module import
from glob import glob       # Creates lists of files for batch processing

# Import third party libraries
import earthpy as et        # Earthpy library
import earthpy.earthexplorer as etee    # Plot and manipulate spatial data
import earthpy.spatial as es    # Used for spatial statistics
import geopandas as gpd     # Work with vector files
import geoviews as gv       # Visualization package
import holoviews as hv      # Visualization package
import hvplot.pandas        # Used for plotting data
import hvplot.xarray        # Used for plotting maps
import numpy as np          # Used for math operations
import pandas as pd         # Used for working with tabular data
import rioxarray as rxr     # Storage and manipulation of rasters
import rioxarray.merge as rxrm   # Merge rioxarrays
import xarray as xr         # Work with multidimensional arrays

# Set up logging so AppeearsDownloader will log in notebook
logging.basicConfig(level=logging.INFO)

# Ignore FutureWarning
# warnings.simplefilter(action='ignore')

# Get data and neighborhood directory paths and save as variables
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
hood_dir = os.path.join(data_dir, 'boise-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'boise-green-space', 'processed')

# If directories do not exist, create
for a_dir in [hood_dir, ndvi_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

ModuleNotFoundError: No module named 'earthpy.earthexplorer'

## Data Description - Neighborhood Boundaries

Polygons of the officially registered neighborhood associations in the city of Boise were used for this analysis. Neighborhood associations are non-profit organizations composed of land owners representing community areas with distinctly prescribed style or character as part of an official planning unit.The data was created by COMPASS, a regional planning agency for Ada and Canyon counties and is maintained by the City of Boise GIS team.

**Citation:**

City of Boise (2018) Neighborhood Associations. https://opendata.cityofboise.org/datasets/9751fb5ec88b47918ec3c0c866609f6d/explore?location=43.599418%2C-116.139637%2C11.42. Accessed 11/30/2023.

In [ ]:
# Access the neighborhood data from the data portal
# Download the city neighborhood shapefile if it does not exist
hood_path = os.path.join(hood_dir, 'neighborhood_associations.shp')

if not os.path.exists(hood_path):
    hood_url = ('https://services1.arcgis.com/WHM6qC35aMtyAAlN/arcgis/'
                'rest/services/BoiseNeighborhoodAssociations/FeatureServer/0/'
                'query?outFields=*&where=1%3D1&f=geojson')
    gpd.read_file(hood_url).to_file(hood_path)

# Load the city neighborhood data into a gdf
hood_bndry = gpd.read_file(hood_path).set_index('ASSOCIATIO')
# hood_bndry


# Select the rows from specific neighborhood boundaries
neigh_bndry = (
    hood_bndry
    .loc[['Central Rim', 'West End']]
)
neigh_bndry

In [ ]:
# Reproject roads and counties to CRS 3857
neigh_bndry_rpj = neigh_bndry.to_crs({'init':'epsg:3857'})

# Print the CRS of each file after reprojection
print('Geodetic CRS of neighborhood boundary before reprojection: ' 
      + str(neigh_bndry.crs))
print('Geodetic CRS of neighborhood boundary after reprojection: ' 
      + str(neigh_bndry_rpj.crs))

neigh_bndry_rpj.hvplot(
    x='x', y='y',
    xlabel='Longitude (decimal degrees)',
    ylabel='Latitude (decimal degrees)',
    title='Boundary of the Select Boise Neighborhoods',
    aspect='equal',
    height=600,
    width=600,
    xformatter='%.0f',
    yformatter='%.0f',
    line_color='red',
    line_width=2,
    fill_color='none',
    tiles='OSM')


### NAIP Multispectral Data

Use multispectral data from the [National Agricuture Imagery Program](https://naip-usdaonline.hub.arcgis.com/) for this analysis. Multispectral imagery can be enhanced using [false color images](https://earthobservatory.nasa.gov/features/FalseColor/page6.php) or [spectral indices]() in order to highlight phenomena such as vegetation health, wetness, or heat. In this analysis, you will produce a color infrared (CIR) false color image as well as a normalized difference vegetation index (NDVI) image. Both of these methods will enhance differences in vegetation health captured by the data.


YOUR TASK: In the cell below, describe the data you use in this analysis, including a citation. 

In [ ]:
def download_neighborhood_data(nname, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date

    Downloads data from the National Agricultural Imagery Program (NAIP)
    given a spatial and temporal extent. 
    Data from: https://naip-usdaonline.hub.arcgis.com/

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """

    print(f'\nNeighborhood name: {nname}')      # Print neigh name  
    bbox = etee.BBox(*geometry.bounds)          # Provide neighborhood bound
    naip_downloader = etee.EarthExplorerDownloader(
          dataset="NAIP",                         # Provide dataset name
          label=nname.lower().replace(' ', '-'),  # Assign a label to the data
          bbox=bbox,                              # Indicate using bound box
          start=start,                            # Data start date
          end=end,                                # Data end date
          store_credential=True)                 # Save login credentials T/F
    naip_downloader.submit_download_request()     # Submit download request
    naip_downloader.download(override=True)

    print(glob(os.path.join(naip_downloader.data_dir, '*.tif'))) 
    return naip_downloader

# Create file for stats and save the path as a variable
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

# Determine if stats file exists - open or create empty df
if os.path.exists(ndvi_stats_path):
  print('Reading in NDVI statistics file..')
  ndvi_stats_df = pd.read_csv(ndvi_stats_path)
else:
   print('NDVI statistics file does not exits...')
   ndvi_stats_df = pd.DataFrame()

# Determine if statistics have been calculated for specified neighborhoods
for neighborhood_name, details in neigh_bndry.iterrows():
    # If statistics exist, skip to next iterration
    if neighborhood_name in ndvi_stats_df.index:
      print (
         'Neighborhood statistics have already been calculated. Skipping...')
      continue
    # If no statistics exist, download data and get stats
    downloader = download_neighborhood_data(
       neighborhood_name, details.geometry,
       '2021-01-01',                                # NAIP Start Date
       '2021-12-31'                                 # NAIP End Date
       )

In [ ]:
def load_and_merge_arrays(nname):
    """
    Load in and merge download arrays

    Parameters
    ==========
    name : str
      The name used to label the download

    Returns
    =======
    merge_da : rxr.DataArray
      DataAray with merged data
    """

    print(f'\nNeighborhood name: {nname}')           # Print neigh name
    data_path = os.path.join(                        # Find dir for each neigh
        et.io.HOME, et.io.DATA_NAME, 
        nname.lower().replace(' ', '-')
        )
    tif_paths = glob(os.path.join(data_path, '*.tif'))  # tif path for neigh
    das = [rxr.open_rasterio(
        tp, masked=True) for tp in tif_paths]        # Open Naip tile
    merge_da = rxrm.merge_arrays(das)           # Merge tiles and save to dict
    return merge_da

merge_da = load_and_merge_arrays('West End')
merge_da

In [ ]:
# Calculate NDVI statistics for all neighborhoods 

def calculate_ndvi_statistics(
      neigh_gdf, naip_full_da, stats_path, override=True):
    """
    Calculate NDVI then summarize and save statistics

    Uses downloaded National Agricultural Imagery Program (NAIP)
    data in the calculations.

    Parameters
    ==========
    neigh_gdf : gpd.GeoDataFrame
      One row with the neighborhood name and boundary geometry
    naip_full_da : rxr.DataArray
      Multispectral (NAIP raster data)
    stats_path : pathlike
      The path to the statistics file to save results
    """

    name = str(neigh_gdf.index[0])
    print(f'\nNeighborhood name: {name}')

    # Determine if CSV file exists and is empty
    file_is_empty = True
    if os.path.exists(stats_path):
      print('\nStatistics file exists.')
      stats_df = pd.read_csv(stats_path)
      file_is_empty = len(stats_df)==0
      print(f'\nStatistics file is empty? {file_is_empty}')

      if not file_is_empty:
        if (name in list(stats_df.neighborhood)) and (not override):
           print('Statistics already calculated. Skipping...')
           return

    # Reproject the data crs
    neigh_rpj = neigh_gdf.to_crs(naip_full_da.rio.crs)
      
    # Clip indiv neigh using box
    naip_clp_da = naip_full_da.rio.clip_box(
        *neigh_rpj.total_bounds)
    
    # Get geometry of indiv neigh
    naip_da = naip_clp_da.rio.clip(
        neigh_rpj.geometry)

    # Calculate NDVI from NAIP
    ndvi_da = (
        (naip_da.sel(band=4) - naip_da.sel(band=1))
        / (naip_da.sel(band=4) + naip_da.sel(band=1))
    )

    # Calculate Spatial Statistics and save to a csv  
    print('\nWriting statistics to file...')
    mode = 'w' if file_is_empty else 'a'   
    pd.DataFrame(dict(
        neighborhood=[name],
        ndvi_min=[float(ndvi_da.min())],                      # Minimum
        ndvi_max=[float(ndvi_da.max())],                      # Maximum
        ndvi_med=[float(ndvi_da.median())],                   # Median
        ndvi_25pctl=[np.nanpercentile(ndvi_da.values, 25)],   # 25th percentile
        ndvi_75pctl=[np.nanpercentile(ndvi_da.values, 75)],   # 75th percentile
        ndvi_mean=[float(ndvi_da.mean())],                    # Mean
        ndvi_std=[float(np.std(ndvi_da))]                     # Standard Dev.
    )).to_csv(
       stats_path, mode=mode, header=file_is_empty, index=False) 

calculate_ndvi_statistics(
    hood_bndry.loc[['West End']], merge_da, ndvi_stats_path)

In [ ]:
# Create file for stats and save the path as a variable
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

# Cycle through all of the city neighborhoods
for neighborhood_name, details in hood_bndry.iterrows():
    # Determine if stats file exists. If not, create an df or else read
    if not os.path.exists(ndvi_stats_path):
        print('NDVI statistics file does not exits...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
    
    if neighborhood_name in ndvi_stats_df.index:
       print(f'Neighborhood name: {neighborhood_name}')
       print(
          'Neighborhood statistics have already been calculated. Skipping...')
       continue

    downloader = download_neighborhood_data(
        neighborhood_name, 
        details.geometry,
        '2021-01-01', '2021-12-31')
    
    merge_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_statistics(
        hood_bndry.loc[[neighborhood_name]], merge_da, ndvi_stats_path)
    
    shutil.rmtree(downloader.data_dir)

In [ ]:
# Create a chloropleth map from NDVI statistics
ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")

(gv.tile_sources.StamenTerrainRetina * gv.Polygons(
    hood_bndry.join(ndvi_stats_df, how='left'),
    vdims=['ndvi_mean'])
 .opts(
    width = 1000, height = 600,
    colorbar=True, cmap='plasma',
    color='Mean NDVI', toolbar='above',
    title= 'Mean NDVI for Neighborhoods in Boise, ID'
 ))
 
# hv.save(chloropleth, 'chloropleth.html')

In [ ]:
%%capture
%%bash
jupyter nbconvert ndvi_fire_CRG.ipynb --to html --no-input